In [1]:
import pandas as pd
import numpy as np
import itertools
import pickle
import modelfunctions as mfunc

In [2]:
folder = "../Forecast/"
with open(f'{folder}fusion_forecast_feed_price.pickle', 'rb') as f:
    fp = pickle.load(f)
with open(f'{folder}fusion_forecast_feed_tons.pickle', 'rb') as f:
    ft = pickle.load(f)
with open(f'{folder}fusion_forecast_prod_price.pickle', 'rb') as f:
    pp = pickle.load(f)
with open(f'{folder}fusion_forecast_prod_tons.pickle', 'rb') as f:
    pt = pickle.load(f)

In [3]:
# Convert dictionary to multi-level DataFrame
def convert_to_df(dict, column_names = ["P", "M", "T"]):
    multi_level_df = pd.concat({(P, M): pd.Series(T) for P, inner_dict in dict.items() for M, T in inner_dict.items()}, axis=0)
    multi_level_df.index.names = column_names

    # Reset the index to get columns 'P', 'M', 'T'
    final_df = multi_level_df.reset_index()
    final_df.columns = column_names + ['Value']
    return final_df

demand = convert_to_df(pt)
demand

,P,M,T,Value
0,LA,BE,2024-01-01,1545.467041
1,LA,BE,2024-02-01,1557.195891
2,LA,BE,2024-03-01,1552.327676
3,LA,BE,2024-04-01,1538.103285
4,LA,BE,2024-05-01,1549.811146
...,...,...,...,...
1075,SA,NL,2033-08-01,1102.088178
1076,SA,NL,2033-09-01,1104.738928
1077,SA,NL,2033-10-01,1106.873724
1078,SA,NL,2033-11-01,1106.940830


In [4]:
def check(dict, df, P = "P", M = "M", T = "T"):
    count = 0
    for p in dict.keys():
        for geo in dict[p].keys():
            dict[p][geo].index.freq = None
            if any(dict[p][geo].rename("Value") !=  df[(df[P] == p) & (df[M] == geo)].set_index(f"{T}").Value):
                count += 1
    if count > 0:
        print(count)
    else:
        print("All good!")
    
check(pt, demand)

All good!


In [5]:
supply = convert_to_df(ft)
check(ft, supply)
supply

All good!


,P,M,T,Value
0,Maize,BE,2024-01-01,1.511336e+06
1,Maize,BE,2024-02-01,1.511336e+06
2,Maize,BE,2024-03-01,1.511336e+06
3,Maize,BE,2024-04-01,1.511336e+06
4,Maize,BE,2024-05-01,1.511336e+06
...,...,...,...,...
1075,Beet,FR,2033-08-01,1.252857e+07
1076,Beet,FR,2033-09-01,1.252857e+07
1077,Beet,FR,2033-10-01,1.252857e+07
1078,Beet,FR,2033-11-01,1.252857e+07


In [6]:
price_p = convert_to_df(pp)
check(pp, price_p)
price_p

All good!


,P,M,T,Value
0,LA,BE,2024-01-01,1504.462288
1,LA,BE,2024-02-01,1491.108591
2,LA,BE,2024-03-01,1477.883113
3,LA,BE,2024-04-01,1469.621768
4,LA,BE,2024-05-01,1443.971842
...,...,...,...,...
1075,SA,NL,2033-08-01,2500.743639
1076,SA,NL,2033-09-01,2503.274438
1077,SA,NL,2033-10-01,2505.810570
1078,SA,NL,2033-11-01,2508.353516


In [7]:
price_f = convert_to_df(fp)
check(fp, price_f)
price_f

All good!


,P,M,T,Value
0,Maize,BE,2024-01-01,195.576575
1,Maize,BE,2024-02-01,194.347345
2,Maize,BE,2024-03-01,196.322190
3,Maize,BE,2024-04-01,199.567966
4,Maize,BE,2024-05-01,202.130520
...,...,...,...,...
1075,Beet,FR,2033-08-01,142.337920
1076,Beet,FR,2033-09-01,142.218417
1077,Beet,FR,2033-10-01,142.103618
1078,Beet,FR,2033-11-01,141.993331


In [8]:
aux = price_f.copy()
aux["T"] = aux["T"].dt.year
aux.groupby(["P", "M", "T"], as_index = False).Value.sum()

,P,M,T,Value
0,Beet,BE,2024,1754.511722
1,Beet,BE,2025,1700.136710
2,Beet,BE,2026,1668.575351
3,Beet,BE,2027,1648.700770
4,Beet,BE,2028,1635.867145
...,...,...,...,...
85,Wheat,FR,2029,3011.310259
86,Wheat,FR,2030,3057.900966
87,Wheat,FR,2031,3107.773474
88,Wheat,FR,2032,3160.523961


In [21]:
def optimization_data(yearly = False):
    dfs = {
        "Price_F": price_f.copy(),
        "Price_P": price_p.copy(),
        "Demand": demand.copy(),
        "Supply": supply.copy(),
    }
    for df in dfs.keys():
        if yearly:
            dfs[df]["T"] = dfs[df]["T"].dt.year
            cols = list(dfs[df].columns)
            cols.remove("Value")

            dfs[df] = dfs[df].groupby(by=cols, as_index = False).Value.sum() if df in ["Demand", "Supply"] else dfs[df].groupby(by=cols, as_index = False).Value.first()

        unique_vals = sorted(dfs[df]['T'].unique())
        l = list(range(1, 1 + len(unique_vals)))

        replace_dict = dict(zip(unique_vals, l))
        dfs[df]["T"] = dfs[df]["T"].replace(replace_dict)

        dfs[df]["P"] = dfs[df]["P"].replace({"LA" : 'Lactic Acid', 'SA': 'Succinic Acid'})

        #dfs[df].to_excel(f"{df}.xlsx")

    with open("forecasts", "wb") as f:
        pickle.dump(dfs, f)
    return dfs


dfs = optimization_data(yearly = True)
dfs["Demand"]

,P,M,T,Value
0,Ethanol,DE,1,1.219012e+06
1,Ethanol,DE,2,1.252122e+06
2,Ethanol,DE,3,1.284971e+06
3,Ethanol,DE,4,1.316157e+06
4,Ethanol,DE,5,1.345963e+06
...,...,...,...,...
85,Succinic Acid,NL,6,1.321831e+04
86,Succinic Acid,NL,7,1.323498e+04
87,Succinic Acid,NL,8,1.324769e+04
88,Succinic Acid,NL,9,1.325808e+04


In [17]:
with open("forecasts", "rb") as f:
    dfs = pickle.load(f)

dfs["Price_F"].Value.max()

349.46756229667164

In [19]:
dfs["Demand"].P.unique()

array(['Ethanol', 'Lactic Acid', 'Succinic Acid'], dtype=object)

In [20]:
dfs["Price_F"]

,P,M,T,Value
0,Beet,BE,1,122.649989
1,Beet,BE,2,117.070449
2,Beet,BE,3,113.260833
3,Beet,BE,4,111.182578
4,Beet,BE,5,109.889548
...,...,...,...,...
85,Wheat,FR,6,249.367837
86,Wheat,FR,7,253.113343
87,Wheat,FR,8,257.150607
88,Wheat,FR,9,261.441226


In [14]:
dfs["Supply"].P.unique()

array(['Beet', 'Maize', 'Wheat'], dtype=object)